In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
def corr2d(X,K):
    #计算二维互相关运算
    h,w=K.shape
    #输出的张量应该比原本的小一圈
    Y=torch.zeros(size=(X.shape[0]-h+1,X.shape[1]-w+1))
    for j in range(Y.shape[1]):
        for i in range(Y.shape[0]):
            #这里的*表示点积
            Y[i,j] =(X[i:i+h,j:j+w]*K).sum()
    
    return Y

In [10]:
X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

# 卷积层
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。 所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。 就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

基于上面定义的 corr2d 函数实现二维卷积层。在 __init__ 构造函数中，将 weight 和 bias 声明为两个模型参数。前向传播函数调用 corr2d 函数并添加偏置。

In [12]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(size=kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
    
    def forward(self,X):
        return corr2d(X,self.weight)+self.bias


# 图像中目标的边缘检测
如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。 首先，我们构造一个  6×8  像素的黑白图像。中间四列为黑色（ 0 ），其余像素为白色（ 1 ）。

In [13]:
X=torch.ones(size=(6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为  1  、宽度为  2  的卷积核 K 。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [24]:
K=torch.tensor([[-1,1]])

现在，我们对参数 X （输入）和 K （卷积核）执行互相关运算。 如下所示，输出Y中的-1代表从白色到黑色的边缘，1代表从黑色到白色的边缘，其他情况的输出为  0 。

In [25]:
Y=corr2d(X,K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘。

In [29]:
#X.t()的意思是转置
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

# 学习卷积核
如果我们只需寻找黑白边缘，那么以上 [1, -1] 的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计过滤器。那么我们是否可以学习由X生成Y的卷积核呢？

现在让我们看看是否可以通过仅查看“输入-输出”对来学习由 X 生成 Y 的卷积核。 我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较 Y 与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [51]:
#构造一个二维的卷积层，它具有一个输出通道和形状为1×2的卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat=conv2d(X)
    loss =(Y_hat-Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    #迭代卷积核
    conv2d.weight.data[:]-=3e-2*conv2d.weight.grad

    print(f'batch {i+1},loss {loss.sum():.3f}')

batch 1,loss 13.202
batch 2,loss 5.737
batch 3,loss 2.561
batch 4,loss 1.184
batch 5,loss 0.571
batch 6,loss 0.289
batch 7,loss 0.154
batch 8,loss 0.086
batch 9,loss 0.050
batch 10,loss 0.030


在  10  次迭代之后，误差已经降到足够低。现在我们来看看我们所学的卷积核的权重张量。

In [52]:
conv2d.weight.data.reshape(shape=(1,2))

tensor([[-0.9720,  1.0052]])